In [ ]:
import cv2
import numpy as np
import os

# Function to detect skin using OpenCV
def detect_skin(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define lower and upper bounds for skin color in HSV space
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    
    # Create a skin mask
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    # Apply the mask to the image
    skin = cv2.bitwise_and(image, image, mask=skin_mask)
    
    return skin, skin_mask

# Folder containing images
input_folder = '../skinType'
output_folder = 'path/to/save/cropped/images/'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Iterate through each image in the folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)

        # Detect skin and apply the mask
        skin, skin_mask = detect_skin(image)

        # Find the bounding box of the skin region
        contours, _ = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            # Get the largest contour (assuming it's the skin)
            largest_contour = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(largest_contour)

            # Crop the skin region from the image
            cropped_skin = image[y:y+h, x:x+w]

            # Save the cropped image (you can keep the original resolution)
            output_path = os.path.join(output_folder, f'cropped_{filename}')
            cv2.imwrite(output_path, cropped_skin)

            print(f"Processed: {filename}")
